In [3]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections

In [4]:
def collect_losses(root_dir):

    depth = 2

    dict_res_roberta = dict()
    dict_res_xlm = dict()
    
    for subdir, dirs, files in os.walk(root_dir):
        if subdir[len(root_dir):].count(os.sep) < depth:
            for file_ in files:
                if file_ == "test_results.json":
                    with open(subdir + "/" + file_) as f:
                        d = json.load(f)
                        test_loss = d["test_loss"]
                        test_accuracy = d["test_accuracy"]
                        
                    model_type = subdir.split(os.sep)[-1].split("_")[0]
                    if model_type == "roberta":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_roberta[dataset_name] = {"loss": test_loss, "accuracy": test_accuracy}
                    elif model_type == "xlm":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_xlm[dataset_name] = {"loss": test_loss, "accuracy": test_accuracy}

    return dict_res_roberta, dict_res_xlm

def average_en_p_rows(s):
    s = s[[" f" in x for x in s.index]]
    s = s[[" en" in x for x in s.index]]
    s = s[[not "np " in x for x in s.index]]
    return s.mean()

def average_en_np_rows(s):
    s = s[[" f" in x for x in s.index]]
    s = s[[all([x in i for x in ["np ", " en"]]) for i in s.index]]
    return s.mean()

def average_it_p_rows(s):
    s = s[[" f" in x for x in s.index]]
    s = s[[" it" in x for x in s.index]]
    s = s[[not "np " in x for x in s.index]]
    return s.mean()

def average_it_np_rows(s):
    s = s[[" f" in x for x in s.index]]
    s = s[[all([x in i for x in ["np ", " it"]]) for i in s.index]]
    return s.mean()


def process_to_present(df_res, column_width="1.1cm", plot_transpose=True):
    # df_res = df_res.rename(columns=mapping_columns_names)
    if not plot_transpose:
        df_res = df_res.T
    s = df_res.style
    s.format(na_rep='MISS', precision=2)
    print(s.to_latex(column_format='l'+('p{'+f'{column_width}'+'}')*len(df_res.columns)))

In [5]:
res_roberta, res_xlm = collect_losses("../output/sst2")

## ROBERTA

In [10]:
df_roberta = pd.DataFrame(res_roberta).T
avg_p = df_roberta.apply(average_en_p_rows)
avg_np = df_roberta.apply(average_en_np_rows)
df_roberta.loc["average p"] = avg_p
df_roberta.loc["average np"] = avg_np

In [11]:
df_roberta

,loss,accuracy
np f en6,0.465598,0.807339
p f en98,0.249827,0.925459
p f en6,0.234029,0.928899
np nf,0.442190,0.806193
p f en83,0.237585,0.931193
np f en57,0.464771,0.805046
p f en49,0.249768,0.926606
p nf,0.232519,0.936927
p f en57,0.236757,0.932339
np f en83,0.456971,0.802752


In [9]:
process_to_present(df_roberta)

\begin{tabular}{lp{1.1cm}p{1.1cm}}
 & loss & accuracy \\
np f en6 & 0.47 & 0.81 \\
p f en98 & 0.25 & 0.93 \\
p f en6 & 0.23 & 0.93 \\
np nf & 0.44 & 0.81 \\
p f en83 & 0.24 & 0.93 \\
np f en57 & 0.46 & 0.81 \\
p f en49 & 0.25 & 0.93 \\
p nf & 0.23 & 0.94 \\
p f en57 & 0.24 & 0.93 \\
np f en83 & 0.46 & 0.80 \\
np f en49 & 0.46 & 0.80 \\
np f en98 & 0.46 & 0.80 \\
average p & 0.24 & 0.93 \\
average np & 0.46 & 0.80 \\
\end{tabular}



## XLM

In [15]:
df_xlm = pd.DataFrame(res_xlm).T
avg_p = df_xlm.apply(average_en_p_rows)
avg_np = df_xlm.apply(average_en_np_rows)
df_xlm.loc["average p"] = avg_p
df_xlm.loc["average np"] = avg_np

In [13]:
df_xlm

,loss,accuracy
np f en49,0.446184,0.788991
p f en83,0.539081,0.743119
p f en49,0.318479,0.889908
np f en83,0.443841,0.788991
en p nf,0.305635,0.916284
p f en98,0.289272,0.887615
np f en98,0.450277,0.799312
np f en6,0.451393,0.793578
en np nf,0.451625,0.793578
p f en57,0.293357,0.897936


In [16]:
process_to_present(df_xlm)

\begin{tabular}{lp{1.1cm}p{1.1cm}}
 & loss & accuracy \\
np f en49 & 0.45 & 0.79 \\
p f en83 & 0.54 & 0.74 \\
p f en49 & 0.32 & 0.89 \\
np f en83 & 0.44 & 0.79 \\
en p nf & 0.31 & 0.92 \\
p f en98 & 0.29 & 0.89 \\
np f en98 & 0.45 & 0.80 \\
np f en6 & 0.45 & 0.79 \\
en np nf & 0.45 & 0.79 \\
p f en57 & 0.29 & 0.90 \\
p f en6 & 0.28 & 0.91 \\
np f en57 & 0.45 & 0.80 \\
average p & 0.34 & 0.87 \\
average np & 0.45 & 0.79 \\
\end{tabular}

